Dataset - "Business Licences"

Objective:
- Prepare dataset for visualization so that analyses about the City of Vancouver's development, with regards to businesses, can be made.

Notes: 
- Data is sourced from the City of Vancouver's Open Data Portal (https://opendata.vancouver.ca)
- Dataset is licensed under the 'Open Government Licence - Vancouver' (https://opendata.vancouver.ca/pages/licence/).

- Original dataset needs to be in CSV format.
- Data includes business licenses from 2013 onwards.
- There is a small possibility that licence numbers are not unique.
- 'LicenceRSN' is meant to be an unique identifier.
- Licences issued to addresses outside of the City of Vancouver do not have coordinates.
- Some City of Vancouver addresses do not have coordinates.
- Some licences are replacement licences.
- Some address data is hidden for privacy protection reasons.
- Liquor licences do not have 'IssueDate' or 'ExpiredDate' values due to being intermin applications for valid business licences.


In [ ]:
import os
import math
import string
import difflib
from pathlib import Path
from random import randrange
from datetime import datetime
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Change Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Functions


def import_data(dataset_name, data_dir_path):

    data_file_paths = _show_data_files(dataset_name, data_dir_path)
    data_file_path = _get_file_index(data_file_paths)

    if data_file_path.suffix == '.json':
        data = pd.read_json(data_file_path)
    elif data_file_path.suffix == '.csv':
        data = pd.read_csv(data_file_path, sep=';')

    return data, data_file_path.name


def _show_data_files(dataset_name, data_dir_path):

    data_file_paths = list(data_dir_path.iterdir())
    relevant_file_paths = [path for path in data_file_paths if dataset_name.lower() in path.name.lower()]

    for index, file_path in enumerate(relevant_file_paths):
        print(f'({index}) {file_path}')

    return relevant_file_paths 


def _get_file_index(data_file_paths):

    while True:
        try:
            data_file_index = int(input('Enter the index of the data file to be imported, e.g. 0: '))
            data_file_path = data_file_paths[data_file_index]
        except (ValueError, IndexError, TypeError):
            print('Please try again with a valid index.')
        else: 
            return data_file_path


def get_metro_van_areas():

    metro_van_membership = pd.read_html('https://en.wikipedia.org/wiki/Metro_Vancouver_Regional_District#Membership')[1]
    metro_van_members = metro_van_membership[0:23]['Member'].values

    return metro_van_members


def show_random_rows(data, n):

    indexes = [randrange(len(data.index)) for _ in range(n)]

    return data.loc[indexes]


def fix_capitalization(value):

    if isinstance(value, str):
        return string.capwords(value)
    else:
        return value


def predict_city(city, possibilities):

    if not isinstance(city, str):
        return city
        
    close_matches = difflib.get_close_matches(city.lower(), possibilities=possibilities, n=1, cutoff=0.8)

    if close_matches:
        return close_matches[0]
    else:
        return city


def convert_province(value, province):

    if str(value).lower() in PROVINCES[province]:
        return province
    return value


def remove_dupes(business_licences):
    dupes_remove = []
    licence_counts = business_licences['LicenceRSN'].value_counts()
    dupe_counts = licence_counts[licence_counts > 1]

    for dupe_idx in dupe_counts.index:
        idxes_remove = _get_idxes_to_remove(business_licences, dupe_idx)
        dupes_remove += idxes_remove

    business_licences.drop(index=dupes_remove, inplace=True)

    return business_licences


def _get_idxes_to_remove(business_licences, dupe_idx):
    idx_keep = None

    dupes = business_licences.loc[business_licences['LicenceRSN'] == dupe_idx]
    dupes.sort_index(inplace=True)
    n_dupes = len(dupes.index)
    idxes = dupes.index.tolist()
    missing_count = dupes.isna().sum(axis=1)

    # Check for least missing values
    least_missing_idx = missing_count[missing_count == missing_count.min()].index.tolist()
    if len(least_missing_idx) == 1:
        idx_keep = least_missing_idx[0]
    # Check for highest index value based on original order of records    
    elif len(least_missing_idx) > 1:
        highest_idx = dupes.index.max()
        idx_keep = highest_idx

    # Create list of indexes to remove
    idxes_remove = [idx for idx in idxes if idx != idx_keep]

    return idxes_remove


def export_data(data, dataset_name, data_file_name, filetype='csv'):
    
    utc_now = str(datetime.utcnow())
    date_time_now = f'{utc_now[0:10]}-{utc_now[11:13]}{utc_now[14:16]}'

    file_name = re.match(rf'\d{{4}}-\d{{2}}-\d{{2}}-\d{{4}}_{dataset_name}', data_file_name).group()
    new_name = f'{file_name}_(processed_{date_time_now}).{filetype}'

    full_path = DATA_DIR_PATH / new_name
    data.to_csv(full_path, sep=';')
    print(f'{new_name} has been successfully exported to {DATA_DIR_PATH}')

In [ ]:
# CONSTANTS

DATASET_NAME = 'business-licences'
DATA_DIR_PATH = Path.cwd() / 'drive' / 'MyDrive' / 'Vancouver Datasets'


# Other Useful Data

# Data saved from Wikipedia
metro_van_members = ['Anmore', 'Belcarra', 'Bowen Island', 'Burnaby', 'Coquitlam', 'Delta',
 'City of Langley', 'Township of Langley', 'Lions Bay', 'Maple Ridge',
 'Metro Vancouver A', 'New Westminster', 'City of North Vancouver',
 'District of North Vancouver', 'Pitt Meadows', 'Port Coquitlam',
 'Port Moody', 'Richmond', 'Surrey', 'Tsawwassen', 'Vancouver',
 'West Vancouver', 'White Rock']

metro_van_cities = np.sort(np.concatenate([metro_van_members, ['Langley', 'Fort Langley', 'North Vancouver']]))

PROVINCES = {'British Columbia': ['bc', 'british columbia'], 
             'Alberta': ['ab', 'alberta'],
             'Saskatchewan': ['sk', 'saskatchewan'],
             'Manitoba': ['mb', 'manitoba'],
             'Ontario': ['on', 'ontario'], 
             'Quebec': ['qc', 'quebec'], 
             'Nova Scotia': ['ns', 'nova scotia'], 
             'New Brunswick': ['nb', 'new brunswick'], 
             'Prince Edward Island': [ 'pei', 'pe', 'prince edward island'], 
             'Newfoundland and Labrador': ['nl', 'newfoundland and labrador'], 
             'Yukon': ['yk', 'yukon'], 
             'Northwest Territories': ['nwt', 'northwest territories'], 
             'Nunavut': ['nt', 'nunavut']}

Import Data

In [ ]:
business_licences, DATA_FILE_NAME = import_data(DATASET_NAME, DATA_DIR_PATH)
business_licences.head()

In [ ]:
business_licences.sort_values('LicenceRSN', inplace=True)
# business_licenses.sort_values('LicenceNumber', inplace=True)

In [ ]:
business_licences.head(5)

Change Data Types

In [ ]:
# BEFORE
print(type(business_licences.at[4, 'IssuedDate']))
print(type(business_licences.at[4, 'ExpiredDate']))
print(type(business_licences.at[4, 'ExtractDate']))
print(business_licences.loc[4, ['IssuedDate', 'ExpiredDate', 'ExtractDate']])

In [ ]:
# Convert date columns into datetime.datetime objects or datetime.date objects
datetime_cols = ['IssuedDate', 'ExtractDate']
date_cols = ['ExpiredDate']

for col in datetime_cols:
    business_licences[col] = pd.to_datetime(business_licences[col], format='%Y-%m-%dT%H:%M:%S.%f', utc=False)
for col in date_cols:
    business_licences[col] = pd.to_datetime(business_licences[col], infer_datetime_format=True).dt.date

Analyze Data - Preliminary

In [ ]:
business_licences.columns

In [ ]:
business_licences.info()

In [ ]:
business_licences.describe()

In [ ]:
business_licences.nunique(dropna=False)

In [ ]:
print(business_licences.isna().sum())
print(f'Number of business licenses: {len(business_licences.index)}')

In [ ]:
business_licences['BusinessType'].unique()

In [ ]:
print(f"Most employees: {business_licences['NumberofEmployees'].max():,.0f}.")
print(f"Mean employees: {business_licences['NumberofEmployees'].mean():,.0f}.")
print(f"Median employees: {business_licences['NumberofEmployees'].median():,.0f}.")
print(f"Least employees: {business_licences['NumberofEmployees'].min():,.0f}.")

Clean Data

In [ ]:
# Rename original 'City' column
business_licences.rename(columns={'City': 'City_(Old)'}, inplace=True)

Clean Data - Correct Misspelt, Duplicate, and Incorrect Data

Reformat and Replace Data - Street

In [ ]:
# Make 'Street' values title case
business_licences['Street'] = business_licences['Street'].apply(fix_capitalization)

Reformat and Replace Data - Postal Code

In [ ]:
# Change 'NOT APPLIC

business_licences.loc[(business_licences['PostalCode'] == 'NOT APPLIC'), 'PostalCode'] = 'Unknown'

Reformat and Replace Data - City

In [ ]:
# Clean and replace 'City' names for Metro Vancouver municipalities.
business_licences['City'] = business_licences['City_(Old)'].apply(predict_city, possibilities=metro_van_cities)

In [ ]:
# Reformat and replace 'City' values that have been missed.

# If 'City' is Langley, assume the Township of Langley due to significantly larger population than the City of Langley.
business_licences.loc[business_licences['City_(Old)'] == 'Langley', 'City'] = 'Township of Langley'

# If 'City' is North Vancouver, assume District of North Vancouver due to larger popualation than the City of North Vancouver.
business_licences.loc[business_licences['City_(Old)'] == 'North Vancouver', 'City'] = 'District of North Vancouver'

business_licences.loc[business_licences['City'] == 'Langley Township', 'City'] = 'Township of Langley'

In [ ]:
# If 'City' is a district and not an actual city, replace with correct city name.
surrey_districts = ['Cloverdale']
langley_city_districts = ['Fort Langley']

business_licences.loc[business_licences['City'].isin(surrey_districts), 'City'] = 'Surrey'
business_licences.loc[business_licences['City'].isin(langley_city_districts), 'City'] = 'City of Langley'

Reformat and Replace Data - Province

In [ ]:
for province in PROVINCES:
    business_licences['Province'] = business_licences['Province'].apply(convert_province, province=province)

business_licences.loc[~business_licences['Province'].isin(PROVINCES), 'Province'] = 'Outside of Canada'

In [ ]:
# CHECK

print(business_licences['Province'].unique())

Reformat and Replace Data - Country

In [ ]:
business_licences['Country'].replace({'CA': 'Canada'}, inplace=True)

Clean Data - Remove Duplicate 'LicenceRSN' Values

In [ ]:
business_licences = remove_dupes(business_licences)

Clean Data - Fill in Missing Data

In [ ]:
# 'LocalArea' only refers to Vancouver districts and confirms that 'City', 'Province', and 'Country' refers to Vancouver, BC, Canada
business_licences.loc[~business_licences['LocalArea'].isna(), 'City'] = 'Vancouver'
business_licences.loc[~business_licences['LocalArea'].isna(), 'Province'] = 'British Columbia'
business_licences.loc[~business_licences['LocalArea'].isna(), 'Country'] = 'Canada'

In [ ]:
# If 'City' is not in Metro Vancouver, change value to 'Outside of Metro Vancouver'
business_licences.loc[~business_licences['City_(Old)'].isin(metro_van_members) & ~(business_licences['City_(Old)'].isna()), 'City']= 'Outside of Metro Vancouver'

# Fill in remaining missing 'City' values
business_licences.loc[business_licences['City_(Old)'].isna(), 'City'] = 'Unknown'

In [ ]:
# Fill in missing 'LocalArea' values
business_licences.loc[(business_licences['City_(Old)'] == 'Vancouver') & (business_licences['LocalArea'].isna()), 'LocalArea'] = 'Unknown Area in Vancouver'
business_licences.loc[(business_licences['City_(Old)'] != 'Vancouver') & 
                      ~(business_licences['City_(Old)'].isna()) & 
                      (business_licences['LocalArea'].isna()) ,'LocalArea'] = 'Outside of Vancouver'
business_licences.loc[(business_licences['City_(Old)'].isna()) & 
                      (business_licences['LocalArea'].isna()) ,'LocalArea'] = 'Unknown'

In [ ]:
# If 'City' has the same name as a Metro Vancouver municipality, assume, for now, 
# it is a Metro Vancouver municipality and not another city of the same name.
business_licences.loc[(business_licences['City_(Old)'].isin(metro_van_members)) & 
                      (business_licences['Province'] == 'British Columbia'), 
                      'Country'] = 'Canada'

business_licences.loc[business_licences['City'].isna(), 'Country'] = 'Unknown'

# Fill in remaining values
business_licences['Country'].fillna('Outside of Canada', inplace=True)

In [ ]:
# Fill in missing 'Country' values
business_licences.loc[~(business_licences['Province'].isin(PROVINCES)), 'Country'] = 'Outside of Canada'

In [ ]:
# Fill in missing 'IssuedDate' values
business_licences.loc[(business_licences['Status'] == 'Issued') & (business_licences['IssuedDate'].isna()), 'IssuedDate'] = 'Unknown'
business_licences.loc[(business_licences['Status'] == 'Cancelled') & (business_licences['IssuedDate'].isna()), 'IssuedDate'] = 'N/A'

In [ ]:
# Fill in missing 'BusinessName' values
business_licences['BusinessName'].fillna('Unknown', inplace=True)

In [ ]:
# CHECK
business_licences.loc[business_licences['BusinessName'].isna(), :]

Clean Data - Checks

In [ ]:
licence_counts = business_licences['LicenceRSN'].value_counts()
dupes = licence_counts[licence_counts > 1]
print(len(dupes.index))
print(dupes[0:10])

In [ ]:
business_licences.loc[(business_licences['City_(Old)'] != 'Vancouver') & (business_licences['Province'] != 'British Columbia'),
                      ['LicenceRSN', 'City_(Old)', 'City']].head(10)

In [ ]:
business_licences.loc[business_licences['Province'] != 'British Columbia', ['City_(Old)', 'City']].head(10)

In [ ]:
unique_cities = business_licences.loc[business_licences['Province'] == 'British Columbia', 'City'].unique()
unique_cities.sort()
print(unique_cities)

In [ ]:
business_licences['Country'].unique()

Clean Data - Cleanup

In [ ]:
business_licences.drop(columns='City_(Old)', inplace=True)

Export Data

In [ ]:
# export_data(business_licences, DATASET_NAME, DATA_FILE_NAME)